<a href="https://colab.research.google.com/github/AdityaSChandra31/Colab_codes/blob/main/MNIST_intro_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
from torchvision import datasets,transforms
from torch.utils.data import DataLoader

#Load MNIST Dataset

train_dataset=datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

test_dataset=datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=transforms.ToTensor()
)


In [12]:
#create DataLoaders

train_loader= DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True
)

test_Loader= DataLoader(
    test_dataset,
    batch_size=1000,
    shuffle=False
)



In [13]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
for batch_idx,(data,target) in enumerate(train_loader):
  data=data.to(device)
  target=target.to(device)

  if batch_idx==0:
    print(f'Input shape:{data.shape}')
    print(f'Targer shape:{target.shape}')
    print(f'Number of training batches:{len(train_loader)}')
    break

Input shape:torch.Size([64, 1, 28, 28])
Targer shape:torch.Size([64])
Number of training batches:938


In [32]:
import torch.nn as nn
import torch.nn.functional as F

class MNIST_MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten= nn.Flatten()
    self.Linear_relu_stack= nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )

  def forward(self,x):
    x=self.flatten(x)
    logits=self.Linear_relu_stack(x)
    return logits

model= MNIST_MLP().to(device)
print(model)

MNIST_MLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (Linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [51]:
criteron=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.00001)

In [39]:
def train_epoch(model,train_loader,optimizer,criterion):
  model.train()
  for batch_idx,(data,target) in enumerate(train_loader):
    data=data.to(device)
    target=target.to(device)

    optimizer.zero_grad()
    output=model(data)
    loss=criterion(output,target)
    loss.backward()
    optimizer.step()

    if batch_idx%64 == 0:
      print(f'Batch {batch_idx}/{len(train_loader)},Loss:{loss.item():.4f}')

train_epoch(model,train_loader,optimizer,criteron)

Batch 0/938,Loss:2.3090
Batch 64/938,Loss:1.4816
Batch 128/938,Loss:0.7429
Batch 192/938,Loss:0.5779
Batch 256/938,Loss:0.4132
Batch 320/938,Loss:0.3300
Batch 384/938,Loss:0.3042
Batch 448/938,Loss:0.4025
Batch 512/938,Loss:0.3257
Batch 576/938,Loss:0.3013
Batch 640/938,Loss:0.3484
Batch 704/938,Loss:0.2416
Batch 768/938,Loss:0.2855
Batch 832/938,Loss:0.2281
Batch 896/938,Loss:0.1126


In [58]:
import torch.nn.functional as F

class LeNet_MNIST(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1= nn.Conv2d(1,6,5)
    self.pool=nn.MaxPool2d(2,2)
    self.conv2=nn.Conv2d(6,16,5)
    self.fc1=nn.Linear(16*4*4,120)
    self.fc2=nn.Linear(120,84)
    self.fc3=nn.Linear(84,10)

  def forward(self,x):
    x=self.pool(F.relu(self.conv1(x)))
    x=self.pool(F.relu(self.conv2(x)))
    x=x.view(-1,16*4*4)
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=self.fc3(x)
    return x

model_lenet=LeNet_MNIST().to(device)
print(model_lenet)


LeNet_MNIST(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [60]:
optimizer_lenet=torch.optim.Adam(model_lenet.parameters(),lr=0.00001)
num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_epoch(model_lenet,train_loader,optimizer_lenet,criteron)

Epoch 1/10
Batch 0/938,Loss:2.1541
Batch 64/938,Loss:2.1164
Batch 128/938,Loss:2.1009
Batch 192/938,Loss:2.1120
Batch 256/938,Loss:2.0566
Batch 320/938,Loss:2.0896
Batch 384/938,Loss:2.0670
Batch 448/938,Loss:1.9846
Batch 512/938,Loss:1.9945
Batch 576/938,Loss:1.9893
Batch 640/938,Loss:1.9797
Batch 704/938,Loss:1.8214
Batch 768/938,Loss:1.8221
Batch 832/938,Loss:1.7441
Batch 896/938,Loss:1.6121
Epoch 2/10
Batch 0/938,Loss:1.7196
Batch 64/938,Loss:1.7765
Batch 128/938,Loss:1.7093
Batch 192/938,Loss:1.5264
Batch 256/938,Loss:1.4789
Batch 320/938,Loss:1.4357
Batch 384/938,Loss:1.4876
Batch 448/938,Loss:1.2587
Batch 512/938,Loss:1.3854
Batch 576/938,Loss:1.3392
Batch 640/938,Loss:1.2961
Batch 704/938,Loss:1.2339
Batch 768/938,Loss:1.2264
Batch 832/938,Loss:1.1954
Batch 896/938,Loss:1.0911
Epoch 3/10
Batch 0/938,Loss:1.0003
Batch 64/938,Loss:0.9584
Batch 128/938,Loss:1.0704
Batch 192/938,Loss:1.0171
Batch 256/938,Loss:0.8980
Batch 320/938,Loss:1.0177
Batch 384/938,Loss:1.0555
Batch 448/938,